# Predicting Residential EV Charging Loads using Neural Networks

In [74]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

## Load, Inspect, and Merge Datasets

### Task 1

The file `'datasets/EV charging reports.csv'` contains electric vehicle (EV) charging data. These come from various residential apartment buildings in Norway. The data includes specific user and garage information, plug-in and plug-out times, charging loads, and the dates of the charging sessions.

Import this CSV file to a pandas DataFrame named `ev_charging_reports`.

Use the `.head()` method to preview the first five rows.

In [75]:
ev_charging_reports = pd.read_csv('Dataset 1_EV charging reports.csv', sep=';')
pd.read_csv
ev_charging_reports['Start_plugin_hour'] = ev_charging_reports['Start_plugin'].apply(lambda x: x[:-3]) + ':00'
ev_charging_reports.head()

,session_ID,Garage_ID,User_ID,User_type,Shared_ID,Start_plugin,Start_plugin_hour,End_plugout,End_plugout_hour,El_kWh,Duration_hours,month_plugin,weekdays_plugin,Plugin_category,Duration_category
0,1,AdO3,AdO3-4,Private,NaN,21.12.2018 10:20,21.12.2018 10:00,21.12.2018 10:23,10.0,"0,3","0,05",Dec,Friday,late morning (9-12),Less than 3 hours
1,2,AdO3,AdO3-4,Private,NaN,21.12.2018 10:24,21.12.2018 10:00,21.12.2018 10:32,10.0,"0,87","0,136666667",Dec,Friday,late morning (9-12),Less than 3 hours
2,3,AdO3,AdO3-4,Private,NaN,21.12.2018 11:33,21.12.2018 11:00,21.12.2018 19:46,19.0,"29,87","8,216388889",Dec,Friday,late morning (9-12),Between 6 and 9 hours
3,4,AdO3,AdO3-2,Private,NaN,22.12.2018 16:15,22.12.2018 16:00,23.12.2018 16:40,16.0,"15,56","24,41972222",Dec,Saturday,late afternoon (15-18),More than 18 hours
4,5,AdO3,AdO3-2,Private,NaN,24.12.2018 22:03,24.12.2018 22:00,24.12.2018 23:02,23.0,"3,62","0,970555556",Dec,Monday,late evening (21-midnight),Less than 3 hours


<details><summary style="display:list-item; font-size:16px; color:blue;">What is the structure of the dataset?</summary>

- **session_ID** - the unique id for each EV charging session
- **Garage_ID** - the unique id for the garage of the apartment
- **User_ID** - the unique id for each user
- **User_private** - 1.0 indicates private charge point spaces and 0.0 indicates shared charge point spaces
- **Shared_ID** - the unique id if shared charge point spaces are used
- **Start_plugin** - the plug-in date and time in the format (day.month.year hour:minute)
- **Start_plugin_hour** - the plug-in date and time rounded to the start of the hour
- **End_plugout** - the plug-out date and time in the format (day.month.year hour:minute)
- **End_plugout_hour** - the start of the hour of the `End_plugout` hour
- **El_kWh** - the charged energy in kWh (charging loads)
- **Duration_hours** - the duration of the EV connection time per session
- **Plugin_category** - the plug-in time categorized by early/late night, morning, afternoon, and evening
- **Duration_category** - the plug-in duration categorized by 3 hour groups
- **month_plugin_{month}** - the month of the plug-in session
- **weekdays_plugin_{day}** - the day of the week of the plug-in session

### Task 2

Import the file `'datasets/Local traffic distribution.csv'` to a pandas DataFrame named `traffic_reports`. This dataset contains the hourly local traffic density counts at 5 nearby traffic locations. 

Preview the first five rows.

In [76]:
traffic_reports = pd.read_csv('Dataset 6_Local traffic distribution.csv', sep=';')
traffic_reports.head()

,Date_from,Date_to,KROPPAN BRU,MOHOLTLIA,SELSBAKK,MOHOLT RAMPE 2,Jonsvannsveien vest for Steinanvegen
0,01.12.2018 00:00,01.12.2018 01:00,639,0,0,4,144
1,01.12.2018 01:00,01.12.2018 02:00,487,153,115,21,83
2,01.12.2018 02:00,01.12.2018 03:00,408,85,75,10,69
3,01.12.2018 03:00,01.12.2018 04:00,282,89,56,8,39
4,01.12.2018 04:00,01.12.2018 05:00,165,64,34,3,25


<details><summary style="display:list-item; font-size:16px; color:blue;">What is the structure of the dataset?</summary>

- **Date_from** - the starting time in the format (day.month.year hour:minute)
- **Date_to** - the ending time in the format (day.month.year hour:minute)
- **Location 1 to 5** - contains the number of vehicles each hour at a specified traffic location.


### Task 3

We'd like to use the traffic data to help our model. The same charging location may charge at different rates depending on the number of cars being charged, so this traffic data might help the model out.

Merge the `ev_charging_reports` and `traffic_reports` datasets together into a Dataframe named `ev_charging_traffic` using the columns:

- `Start_plugin_hour` in `ev_charging_reports`
- `Date_from` in `traffic_reports`

In [77]:
ev_charging_traffic = ev_charging_reports.merge(traffic_reports, 
                                left_on='Start_plugin_hour', 
                                right_on='Date_from')

ev_charging_traffic.head()

,session_ID,Garage_ID,User_ID,User_type,Shared_ID,Start_plugin,Start_plugin_hour,End_plugout,End_plugout_hour,El_kWh,...,weekdays_plugin,Plugin_category,Duration_category,Date_from,Date_to,KROPPAN BRU,MOHOLTLIA,SELSBAKK,MOHOLT RAMPE 2,Jonsvannsveien vest for Steinanvegen
0,1,AdO3,AdO3-4,Private,NaN,21.12.2018 10:20,21.12.2018 10:00,21.12.2018 10:23,10.0,"0,3",...,Friday,late morning (9-12),Less than 3 hours,21.12.2018 10:00,21.12.2018 11:00,3244,1632,545,194,622
1,2,AdO3,AdO3-4,Private,NaN,21.12.2018 10:24,21.12.2018 10:00,21.12.2018 10:32,10.0,"0,87",...,Friday,late morning (9-12),Less than 3 hours,21.12.2018 10:00,21.12.2018 11:00,3244,1632,545,194,622
2,3,AdO3,AdO3-4,Private,NaN,21.12.2018 11:33,21.12.2018 11:00,21.12.2018 19:46,19.0,"29,87",...,Friday,late morning (9-12),Between 6 and 9 hours,21.12.2018 11:00,21.12.2018 12:00,3605,1691,605,230,771
3,4,AdO3,AdO3-2,Private,NaN,22.12.2018 16:15,22.12.2018 16:00,23.12.2018 16:40,16.0,"15,56",...,Saturday,late afternoon (15-18),More than 18 hours,22.12.2018 16:00,22.12.2018 17:00,3052,1484,453,224,694
4,5,AdO3,AdO3-2,Private,NaN,24.12.2018 22:03,24.12.2018 22:00,24.12.2018 23:02,23.0,"3,62",...,Monday,late evening (21-midnight),Less than 3 hours,24.12.2018 22:00,24.12.2018 23:00,1390,693,226,83,353


### Task 4

Use `.info()` to inspect the merged dataset. Specifically, pay attention to the data types and number of missing values in each column.

In [78]:
ev_charging_traffic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6878 entries, 0 to 6877
Data columns (total 22 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   session_ID                            6878 non-null   int64  
 1   Garage_ID                             6878 non-null   object 
 2   User_ID                               6878 non-null   object 
 3   User_type                             6878 non-null   object 
 4   Shared_ID                             1412 non-null   object 
 5   Start_plugin                          6878 non-null   object 
 6   Start_plugin_hour                     6878 non-null   object 
 7   End_plugout                           6844 non-null   object 
 8   End_plugout_hour                      6844 non-null   float64
 9   El_kWh                                6878 non-null   object 
 10  Duration_hours                        6844 non-null   object 
 11  month_plugin     

<details><summary style="display:list-item; font-size:16px; color:blue;">What do we notice about merged dataset under inspection?</summary>

We see that there are 39 columns and 6,833 rows in our merged dataset.

Some notable things we might have to address:

- We expected columns like `El_kWh` and `Duration_hours` to be floats but they are actually object data types.

- There are many identifying columns like `session_ID` and `User_ID` that might not be useful for training.

## Task Group 2 - Data Cleaning and Preparation

### Task 5

Let's start by reducing the size of our dataset by dropping columns that won't be used for training. These include
- ID columns
- columns with lots of missing data
- non-numeric columns (for now, since we haven't yet covered using non-numeric data in neural networks)

Drop columns you don't want to use in training from `ev_charging_traffic_hourly`.

To match our solution, drop the columns

```py
['session_ID', 'Garage_ID', 'User_ID', 
                'Shared_ID',
                'Plugin_category','Duration_category', 
                'Start_plugin', 'Start_plugin_hour', 'End_plugout', 'End_plugout_hour', 
                'Date_from', 'Date_to']
```

In [79]:
ev_charging_traffic_hourly = ev_charging_traffic.drop(columns=['session_ID', 'Garage_ID', 'User_ID', 
                'Shared_ID',
                'Plugin_category','Duration_category', 
                'Start_plugin', 'Start_plugin_hour', 'End_plugout', 'End_plugout_hour', 
                'Date_from', 'Date_to'])
ev_charging_traffic_hourly.head()

,User_type,El_kWh,Duration_hours,month_plugin,weekdays_plugin,KROPPAN BRU,MOHOLTLIA,SELSBAKK,MOHOLT RAMPE 2,Jonsvannsveien vest for Steinanvegen
0,Private,"0,3","0,05",Dec,Friday,3244,1632,545,194,622
1,Private,"0,87","0,136666667",Dec,Friday,3244,1632,545,194,622
2,Private,"29,87","8,216388889",Dec,Friday,3605,1691,605,230,771
3,Private,"15,56","24,41972222",Dec,Saturday,3052,1484,453,224,694
4,Private,"3,62","0,970555556",Dec,Monday,1390,693,226,83,353


### Task 6

Earlier we saw that the `El_kWh` and `Duration_hours` columns were object data types. Upon further inspection, we see that the reason is that the data is following European notation where commas `,` are used as decimals instead of periods.

Replace `,` with `.` in these three columns.

In [80]:
ev_charging_traffic_hourly['El_kWh'] = ev_charging_traffic_hourly['El_kWh'].str.replace(',', '.')
ev_charging_traffic_hourly['Duration_hours'] = ev_charging_traffic_hourly['Duration_hours'].str.replace(',', '.')
ev_charging_traffic_hourly.head()

,User_type,El_kWh,Duration_hours,month_plugin,weekdays_plugin,KROPPAN BRU,MOHOLTLIA,SELSBAKK,MOHOLT RAMPE 2,Jonsvannsveien vest for Steinanvegen
0,Private,0.3,0.05,Dec,Friday,3244,1632,545,194,622
1,Private,0.87,0.136666667,Dec,Friday,3244,1632,545,194,622
2,Private,29.87,8.216388889,Dec,Friday,3605,1691,605,230,771
3,Private,15.56,24.41972222,Dec,Saturday,3052,1484,453,224,694
4,Private,3.62,0.970555556,Dec,Monday,1390,693,226,83,353


### Task 7

Next, convert the data types of all the columns of `ev_charging_traffic` to floats.

In [81]:
weekday_ohe = pd.get_dummies(ev_charging_traffic_hourly.weekdays_plugin, drop_first=False)
month_ohe = pd.get_dummies(ev_charging_traffic_hourly.month_plugin, drop_first=False)
ev_charging_traffic_hourly = ev_charging_traffic_hourly.join([month_ohe, weekday_ohe])
ev_charging_traffic_hourly.drop(columns=['month_plugin', 'weekdays_plugin'], inplace=True)
ev_charging_traffic_hourly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6878 entries, 0 to 6877
Data columns (total 27 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   User_type                             6878 non-null   object
 1   El_kWh                                6878 non-null   object
 2   Duration_hours                        6844 non-null   object
 3   KROPPAN BRU                           6878 non-null   object
 4   MOHOLTLIA                             6878 non-null   object
 5   SELSBAKK                              6878 non-null   object
 6   MOHOLT RAMPE 2                        6878 non-null   int64 
 7   Jonsvannsveien vest for Steinanvegen  6878 non-null   int64 
 8   Apr                                   6878 non-null   uint8 
 9   Aug                                   6878 non-null   uint8 
 10  Dec                                   6878 non-null   uint8 
 11  Feb                           

In [82]:
ev_charging_traffic_hourly.User_type.unique()

array(['Private', 'Shared'], dtype=object)

In [83]:
ev_charging_traffic_hourly['User_private'] = np.where(ev_charging_traffic_hourly['User_type'] == 'Private', 1.0, 0.0)
ev_charging_traffic_hourly.drop(columns=['User_type'], inplace=True)

In [84]:
ev_charging_traffic_hourly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6878 entries, 0 to 6877
Data columns (total 27 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   El_kWh                                6878 non-null   object 
 1   Duration_hours                        6844 non-null   object 
 2   KROPPAN BRU                           6878 non-null   object 
 3   MOHOLTLIA                             6878 non-null   object 
 4   SELSBAKK                              6878 non-null   object 
 5   MOHOLT RAMPE 2                        6878 non-null   int64  
 6   Jonsvannsveien vest for Steinanvegen  6878 non-null   int64  
 7   Apr                                   6878 non-null   uint8  
 8   Aug                                   6878 non-null   uint8  
 9   Dec                                   6878 non-null   uint8  
 10  Feb                                   6878 non-null   uint8  
 11  Jan              

In [85]:
for c in ev_charging_traffic_hourly.columns:
    if ev_charging_traffic_hourly[c].dtype == 'object':
        ev_charging_traffic_hourly[c] = ev_charging_traffic_hourly[c].str.replace('-', '0')
        ev_charging_traffic_hourly[c] = ev_charging_traffic_hourly[c].replace(to_replace='', value='0')
        ev_charging_traffic_hourly[c] = ev_charging_traffic_hourly[c].fillna(0)

In [86]:
for column in ev_charging_traffic_hourly.columns:
    ev_charging_traffic_hourly[column] = ev_charging_traffic_hourly[column].astype(float)

ev_charging_traffic_hourly.head()

,El_kWh,Duration_hours,KROPPAN BRU,MOHOLTLIA,SELSBAKK,MOHOLT RAMPE 2,Jonsvannsveien vest for Steinanvegen,Apr,Aug,Dec,...,Oct,Sep,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,User_private
0,0.30,0.050000,3244.0,1632.0,545.0,194.0,622.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.87,0.136667,3244.0,1632.0,545.0,194.0,622.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,29.87,8.216389,3605.0,1691.0,605.0,230.0,771.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,15.56,24.419722,3052.0,1484.0,453.0,224.0,694.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,3.62,0.970556,1390.0,693.0,226.0,83.0,353.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


## Task Group 3 - Train Test Split

Next, let's split the dataset into training and testing datasets. 

The training data will be used to train the model and the testing data will be used to evaluate the model.

### Task 8

First, create two datasets from `ev_charging_traffic`:

- `X` contains only the input numerical features
- `y` contains only the target column `El_kWh`

In [87]:
X = ev_charging_traffic_hourly.drop(columns=['El_kWh'])
y = ev_charging_traffic_hourly['El_kWh']

### Task 9

Use `sklearn` to split `X` and `y` into training and testing datasets. The training set should use 80% of the data. Set the `random_state` parameter to `2`.

In [88]:
from sklearn.model_selection import train_test_split

C:\Users\Rogo\miniconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [90]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = .8, random_state = 2)

## Task Group 4 - Linear Regression Baseline

This section is optional, but useful. The idea is to compare our neural network to a basic linear regression. After all, if a basic linear regression works just as well, there's no need for the neural network!

If you haven't done linear regression with scikit-learn before, feel free to use [our solution code](./solutions.html) or to skip ahead.

### Task 10

Use Scikit-learn to train a Linear Regression model using the training data to predict EV charging loads.

The linear regression will be used as a baseline to compare against the neural network we will train later.

In [94]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [92]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

### Task 11

Evaluate the linear regression baseline by calculating the MSE on the testing data. Use `mean_squared_error` from `sklearn.metrics`.

Save the testing MSE to the variable `test_mse` and print it out.

In [97]:
y_train_pred = lr_model.predict(X_train)
y_test_pred = lr_model.predict(X_test)

train_MSE = mean_squared_error(y_train, y_train_pred)
test_MSE = mean_squared_error(y_test, y_test_pred)
print(train_MSE)
print(test_MSE)

124.68262070233907
118.94288452322971


Looks like our mean squared error is around `131.4` (if you used different columns in your model than we did, you might have a different value). Remember, this is squared error. If we take the square root, we have about `11.5`. One way of interpreting this is to say that the linear regression, on average, is off by `11.5 kWh`.

## Task Group 5 - Train a Neural Network Using PyTorch

Let's now create a neural network using PyTorch to predict EV charging loads.

### Task 12

First, we'll need to import the PyTorch library and modules.

Import the PyTorch library `torch`.

From `torch`, import `nn` to access built-in code for constructing networks and defining loss functions.

From `torch`, import `optim` to access built-in optimizer algorithms.

### Task 13

Before training the neural network, convert the training and testing sets into PyTorch tensors and specify `float` as the data type for the values.

In [98]:
# Convert training set
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float).view(-1,1)

# Convert testing set
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float).view(-1,1)

### Task 14

Next, let's use `nn.Sequential` to create a neural network.

First, set a random seed using `torch.manual_seed(42)`.

Then, create a sequential neural network with the following architecture:

- input layer with number of nodes equal to the number of training features
- a first hidden layer with `56` nodes and a ReLU activation
- a second hidden layer with `26` nodes and a ReLU activation
- an output layer with `1` node

Save the network to the variable `model`.

In [100]:
X_train_tensor.shape

torch.Size([5502, 26])

In [101]:
torch.manual_seed(42)
model = nn.Sequential(nn.Linear(26,56),
        nn.ReLU(),
        nn.Linear(56,26),
        nn.ReLU(),
        nn.Linear(26,1)
        )

### Task 15

Next, let's define the loss function and optimizer used for training:

- set the MSE loss function to the variable `loss`
- set the Adam optimizer to the variable `optimizer` with a learning rate of `0.0007`

In [103]:
loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0007)

### Task 16

Create a training loop to train our neural network for 3000 epochs.

Keep track of the training loss by printing out the MSE every 500 epochs.

In [108]:
for i in range(3000):
    outputs = model(X_train_tensor)
    mse = loss(outputs, y_train_tensor)
    mse.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (i + 1) % 500 == 0:
        print(f'Epoch {i + 1}: MSE Loss: {mse.item()}')

Epoch 500: MSE Loss: 156.6389617919922
Epoch 1000: MSE Loss: 137.49032592773438
Epoch 1500: MSE Loss: 131.27447509765625
Epoch 2000: MSE Loss: 121.94830322265625
Epoch 2500: MSE Loss: 116.9332046508789
Epoch 3000: MSE Loss: 117.2999496459961


### Task 17

Save the neural network in the `models` directory using the path `models/model.pth`.

In [110]:
torch.save(model, 'model.pth')

### Task 18

Evaluate the neural network on the testing set. 

Save the testing data loss to the variable `test_loss` and use `.item()` to extract and print out the loss. 

In [111]:
test_outputs = model(X_test_tensor)
test_loss = loss(test_outputs, y_test_tensor)
test_loss.item()

114.16789245605469

### Task 19

We trained this same model for 4500 epochs locally. That model is saved as `models/model4500.pth`. Load this model using PyTorch and evaluate it. How well does the longer-trained model perform?

Pretty cool! The increased training improved our test loss to about `115.2`, a full `12%` improvement on our linear regression baseline. So the nonlinearity introduced by the neural network actually helped us out.

That's the end of our project on predicting EV charging loads! Feel free to continue experimenting with this neural network model. 

Some things you might want to investigate further include:
- explore different ways to clean and prepare the data
- we added traffic data, but there's no guarantee that more data converts to a better model. Test out different sets of input columns.
- test out different number of nodes in the hidden layers, activation functions, and learning rates
- train on a larger number of epochs 